In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import time

In [2]:
scrapping_results={} # create a dictionary to hold all results for mongodb

# NASA Mars News

In [3]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 76


In [ ]:
nasa_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
latest_news_title=soup.find('div',{"class":"content_title"}).a.text.strip()

In [ ]:
print(latest_news_title)

In [ ]:
scrapping_results['latest_news_title']=latest_news_title

In [ ]:
latest_news_p=soup.find('div',{"class":"rollover_description_inner"}).text # have to choose this tag, <div class="article_teaser_body"> retuns a notype error

In [ ]:
print(latest_news_p)

In [ ]:
browser.quit()

In [ ]:
scrapping_results['latest_news_p']=latest_news_p

# JPL Mars Featured Image

In [ ]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)

In [ ]:
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
browser.click_link_by_partial_text('FULL IMAGE') # click on FULL IMAGE to go to next page 
time.sleep(5) # wait for 5 seconds until everything is loaded into the 2nd page
browser.click_link_by_partial_text('more info') #click on 'more info' to get the image of original size
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
result=soup.find('figure',{'class':'lede'}).a['href']

In [ ]:
print(result)

In [ ]:
jpl_base_url='https://www.jpl.nasa.gov/'
jpl_featured_image_url=jpl_base_url+result
print(jpl_featured_image_url)

In [ ]:
scrapping_results['jpl_img']=jpl_featured_image_url

In [ ]:
browser.quit()

# Mars Weather

In [ ]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)
twitter_url='https://twitter.com/MarsWxReport'
browser.visit(twitter_url)
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
# since the latest tweet is not always about weather, extract all tweets and use list comprehension to store them in a list
# retrive all <p> tags 
results=soup.find_all('p',{'class':'TweetTextSize'})

In [ ]:
# for some tweets, an achor tag<a> is inside <p> tag. loop through every <p> to find the unwanted <a>, use .extract() to exclude it
# not every <p> has unwanted <a> (AttributedError), have to handle it 
try:
    for tweet in results:
        unwanted=tweet.find('a')
        unwanted.extract()
except AttributeError:
        print('ok')

In [ ]:
#since all unwanted tags removed, retrieve text in <p>
tweet_list=[tweet.text.strip() for tweet in results] 

In [ ]:
# tweets of mars weather always start with 'InSight sol', loop through all tweets, if 'Insight sol' is in a tweet, store it (it's the lastest tweet of mars weather) in a variable and stop the loop immediately
for tweet in tweet_list:
    if ('InSight sol' in tweet):
        mars_weather=tweet.lstrip('InSight ')
        break
print(mars_weather)

In [ ]:
scrapping_results['latest_tweet']=mars_weather

In [ ]:
browser.quit()

# Mars Facts

In [ ]:
# can't use pandas to extract the table here. i tried it only gives the comparison table.
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)
mars_facts_url='https://space-facts.com/mars/'
browser.visit(mars_facts_url)
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
# extract the content of each column and save to a dataframe
first_column=soup.find_all('td',{'class':'column-1'}) # return a list
first_column_content=[i.text for i in first_column]
second_column=soup.find_all('td',{'class':'column-2'})
second_column_content=[i.text for i in second_column]
table_df=pd.DataFrame({'Description':first_column_content,'Value':second_column_content})
mars_facts_html_table=table_df.to_html(index=False,justify='center') # remove index column and put everything in center

In [ ]:
scrapping_results['table']=mars_facts_html_table

# Mars Hemispheres

In [ ]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [ ]:
titles=soup.find_all('h3') #all titles in <h3>tag
title_list=[title.text for title in titles] # extract titles and save to a list
print(title_list)

In [ ]:
usgs_base_url='https://astrogeology.usgs.gov'
next_page_urls=soup.find_all('div',{'class':'description'}) # extract the links to the 2nd page where original img link can be scrapped
next_page_full_url_list=[usgs_base_url+url.a['href'] for url in next_page_urls] 
print(next_page_full_url_list)

In [ ]:
original_image_url_list=[]

In [ ]:
# use loop to visit the 2nd page of each image
for url in next_page_full_url_list:
    browser.visit(url)
    html=browser.html
    soup=bs(html,'html.parser')
    url=soup.find_all('li') # returns a list
    original_image_url_list.append(url[0].a['href']) # there are 2 two links, sample and orginal. the link to sample is the 1st element in the list
print(original_image_url_list)

In [ ]:
browser.quit()

In [ ]:
hemisphere_image_urls=[dict(title=title_list[i].replace(' Enhanced',''), img_url=original_image_url_list[i]) for i in range(4)]
# dict() is a function to create a dictionary
# can't use for loop and list.append, cuz it results in a list of the same elements


In [ ]:
print(hemisphere_image_urls)

In [ ]:
scrapping_results['hemisphere']=hemisphere_image_urls

In [ ]:
print(scrapping_results)